# Ground Truth generation

# table of content
1) [Load stats](#load-stats)
2) [Show histograms and barplots](#show-histograms-and-barplots)
3) [Pie on heights](#pie-on-heights)

### Dependencies and general utils

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d
import laspy
import pdal
import json
from tqdm import tqdm

### Generation

In [5]:
# Loading sources
src_folder_instances = r"..\data\full_dataset\selection\clusters_4\cluster_1\gt\pcd\modified_samples"
src_original_prediction = r"..\data\full_dataset\selection\clusters_4\cluster_1\color_grp_full_tile_316.laz"
src_folder_result = r"..\data\full_dataset\selection\clusters_4\gt"

In [6]:
# Load original and reset/create gt columns
full_tile = laspy.read(src_original_prediction)
# full_tile.remove_extra_dim('gt_semantic_segmentation')
full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_semantic_segmentation',type="uint16"))
full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_instance_segmentation',type="uint16"))
print(full_tile.x)

<ScaledArrayView([2572934.4  2572934.65 2572934.5  ... 2573031.03 2573030.49 2573030.93])>


In [11]:
# Loop on gt instances and set the correct values in the full tile
list_instances_src = [x for x in os.listdir(src_folder_instances) if x.endswith('.laz')]
rounding = 2
semantic_layer = np.zeros(len(full_tile))
instance_layer = np.zeros(len(full_tile))
for id_instance, instance_src in tqdm(enumerate(list_instances_src), total=len(list_instances_src)):
    instance = laspy.read(os.path.join(src_folder_instances, instance_src))
    coords = list(zip(np.round(instance.x, rounding), np.round(instance.y, rounding), np.round(instance.z, rounding)))
    mask = np.array([(x,y,z) in coords for x, y, z in zip(np.round(full_tile.x, rounding), np.round(full_tile.y, rounding), np.round(full_tile.z, rounding))])
    semantic_layer[mask] = 1
    instance_layer[mask] = id_instance + 1
    # print(np.sum(mask))
    # print(len(coords))
    # assert np.sum(mask) == len(coords)

setattr(full_tile, 'gt_semantic_segmentation', semantic_layer)
setattr(full_tile, 'gt_instance_segmentation', instance_layer)

# save file
new_file = os.path.join(os.path.join(src_folder_result), os.path.basename(src_original_prediction).split('.laz')[0] + '_gt.laz')
full_tile.write(new_file)

143


100%|██████████| 143/143 [1:01:11<00:00, 25.67s/it]


### Addition

In [12]:
# Loading sources
src_folder_instances = r"..\data\full_dataset\selection\clusters_4\cluster_1\gt\round2\pcd\modified_samples"
src_target = r"D:\PDM_repo\Github\PDM\data\full_dataset\selection\clusters_4\gt\color_grp_full_tile_316_gt.laz"
# src_original_prediction = r"..\data\full_dataset\selection\clusters_4\cluster_1\color_grp_full_tile_316.laz"
# src_folder_result = r"..\data\full_dataset\selection\clusters_4\gt"
# Load original and reset/create gt columns
tile_target = laspy.read(src_target)

assert "gt_semantic_segmentation" in tile_target.point_format.dimension_names
assert "gt_instance_segmentation" in tile_target.point_format.dimension_names
# # full_tile.remove_extra_dim('gt_semantic_segmentation')
# full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_semantic_segmentation',type="uint16"))
# full_tile.add_extra_dim(laspy.ExtraBytesParams('gt_instance_segmentation',type="uint16"))
# print(full_tile.x)

In [14]:
# Loop on gt instances and set the correct values in the full tile
list_instances_src = [x for x in os.listdir(src_folder_instances) if x.endswith('.laz')]
rounding = 2
semantic_layer = np.array(tile_target.gt_semantic_segmentation)
instance_layer = np.array(tile_target.gt_instance_segmentation)
# instance_layer = np.zeros(len(tile_target))
instance_val = np.max(tile_target.gt_instance_segmentation) + 1
for id_instance, instance_src in tqdm(enumerate(list_instances_src), total=len(list_instances_src)):
    instance = laspy.read(os.path.join(src_folder_instances, instance_src))
    coords = list(zip(np.round(instance.x, rounding), np.round(instance.y, rounding), np.round(instance.z, rounding)))
    mask = np.array([(x,y,z) in coords for x, y, z in zip(np.round(tile_target.x, rounding), np.round(tile_target.y, rounding), np.round(tile_target.z, rounding))])
    semantic_layer[mask] = 1
    instance_layer[mask] = instance_val
    instance_val += 1
    # print(np.sum(mask))
    # print(len(coords))
    # assert np.sum(mask) == len(coords)

setattr(tile_target, 'gt_semantic_segmentation', semantic_layer)
setattr(tile_target, 'gt_instance_segmentation', instance_layer)


100%|██████████| 9/9 [02:03<00:00, 13.75s/it]


In [17]:

# save file
new_file = os.path.join(os.path.join(src_folder_result), os.path.basename(src_original_prediction).split('.laz')[0] + '_gt_2.laz')
tile_target.write(new_file)

In [4]:
# verify results and save file
print("Semantic results:")
for cat in set(full_tile.gt_semantic_segmentation):
    print(f"\tVal {cat}: number of points = {len(full_tile.gt_semantic_segmentation[full_tile.gt_semantic_segmentation == cat])}")
print("Instances results:")
for cat in set(full_tile.gt_instance_segmentation):
    print(f"\tVal {cat}: number of points = {len(full_tile.gt_instance_segmentation[full_tile.gt_instance_segmentation == cat])}")

Semantic results:


AttributeError: LasData object has no attribute 'gt_semantic_segmentation'

In [11]:

for col in list(full_tile.point_format.dimension_names):
    print(col)

X
Y
Z
intensity
return_number
number_of_returns
synthetic
key_point
withheld
overlap
scanner_channel
scan_direction_flag
edge_of_flight_line
classification
user_data
scan_angle
point_source_id
gps_time
red
green
blue
PredSemantic
gt_semantic_segmentation
PredInstance
gt_instance_segmentation


### Introducing clean samples into original

In [ ]:
src_origina